In [ ]:
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import math
import pathlib
from PIL import Image, ImageDraw

In [ ]:
from tensorflow.keras.layers import InputLayer, Input
from tensorflow.keras.layers import MaxPooling2D, Dropout
from tensorflow.keras.layers import Conv2D, Dense, Flatten
from tensorflow.keras.applications import Xception
from tensorflow.keras.applications import xception
from tensorflow.keras.preprocessing.image import img_to_array, load_img, array_to_img
from tensorflow.keras import backend as K


In [ ]:
# в данном решении необходимые файлы хранятся на Google Drive
# получить доступ к Google Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# сохранить пути до датасетов
train_dataset_dir = pathlib.Path(
    "/content/drive/My Drive/cats_dogs_dataset/train")
valid_dataset_dir = pathlib.Path(
    "/content/drive/My Drive/cats_dogs_dataset/valid")

In [ ]:
# сохранить и отсортировать пути до каждой картинки и каждого текстового файла
train_images_paths = list(train_dataset_dir.glob('*.jpg'))
train_labels_paths = list(train_dataset_dir.glob('*.txt'))
valid_images_paths = list(valid_dataset_dir.glob('*.jpg'))
valid_labels_paths = list(valid_dataset_dir.glob('*.txt'))

train_images_paths.sort()
train_labels_paths.sort()
valid_images_paths.sort()
valid_labels_paths.sort()

In [ ]:
# сохранить значения длины каждого списка из путей
t_img_len = len(train_images_paths)
t_txt_len = len(train_labels_paths)
v_img_len = len(valid_images_paths)
v_txt_len = len(valid_labels_paths)

In [ ]:
# создание массива для хранения тренировочных категорических векторов классов
train_class_array = np.zeros([t_txt_len, 2], dtype=('float64'))

# создание массива для хранения тренировочных координат bounding box'а
train_coord_array = np.empty([t_txt_len, 4], dtype=('float64'))

# заполнение массивов для хранения тренировочных координат и классов
for i in range(t_txt_len):

# создание категорического вектора классов
# первая позиция - кошка, вторая - собака
  train_class_array[i][np.loadtxt(train_labels_paths[i], dtype=int)[0]-1] = 1

# заполнение массива для хранения координат bounding box'а
  train_coord_array[i] = np.loadtxt(train_labels_paths[i])[1:]

# создание массива для хранения валидационных категорических векторов классов
valid_class_array = np.zeros([v_txt_len, 2], dtype=('float64'))

# создание массива для хранения валидационных координат bounding box'а
valid_coord_array = np.empty([v_txt_len, 4], dtype=('float64'))

# заполнение массивов для хранения валидационных координат и классов
for i in range(v_txt_len):

# создание категорического вектора классов
# первая позиция - кошка, вторая - собака
  valid_class_array[i][np.loadtxt(valid_labels_paths[i], dtype=int)[0]-1] = 1

# заполнение массива для хранения координат bounding box'а
  valid_coord_array[i] = np.loadtxt(valid_labels_paths[i])[1:]

In [ ]:
# создание массива для хранения тренировочных изображений 256х256
train_img_arr = np.zeros((t_img_len, 256, 256, 3), dtype=('float64'))

# создание массива для множителей изменения размера тренировочных изображений
# для возможности вывода bounding box'а на исходных изображениях
# 256 * множитель = изначальный размер изображения
# первая позиция - x, вторая - y
t_resize_factor_array = np.zeros((t_img_len, 2), dtype=('float64'))

# заполнение массива для хранения тренировочных изображений 256х256
for i in range(t_img_len):

# запись тренировочного изображения в переменную
  jpg_file = load_img(train_images_paths[i])

# расчет множителей изменения размера тренировочных изображений
  t_resize_factor_array[i][0] = jpg_file.size[0]/256 # x
  t_resize_factor_array[i][1] = jpg_file.size[1]/256 # y

# превращение тренировочных координат в относительные, т.е. в промежутке [0,1]
  train_coord_array[i][0] = (
      train_coord_array[i][0]/jpg_file.size[0]
  )
  train_coord_array[i][1] = (
      train_coord_array[i][1]/jpg_file.size[1]
  )
  train_coord_array[i][2] = (
      train_coord_array[i][2]/jpg_file.size[0]
  )
  train_coord_array[i][3] = (
      train_coord_array[i][3]/jpg_file.size[1]
  )

# изменение размера картинки и превращение её в массив numpy
  jpg_file_arr = img_to_array(jpg_file.copy().resize((256, 256)))

# заполнение массива для хранения тренировочных изображений 256х256
  train_img_arr[i] = xception.preprocess_input(jpg_file_arr)

In [ ]:
# создание массива для хранения валидационных изображений 256х256
valid_img_arr = np.zeros((v_img_len, 256, 256, 3), dtype=('float64'))

# создание массива для множителей изменения размера валидационных изображений
# для возможности вывода bounding box'а на исходных изображениях
# 256 * множитель = изначальный размер изображения
# первая позиция - x, вторая - y
v_res_to_orig_multipliers_array = np.zeros((v_img_len, 2), dtype=('float64'))

# заполнение массива для хранения валидационных изображений 256х256
for i in range(v_img_len):

# запись валидационных изображения в переменную
  jpg_file = load_img(valid_images_paths[i])

# расчет множителей изменения размера валидационных изображений
  v_res_to_orig_multipliers_array[i][0] = jpg_file.size[0]/256
  v_res_to_orig_multipliers_array[i][1] = jpg_file.size[1]/256

# превращение валидационных координат в относительные, т.е. в промежутке [0,1]
  valid_coord_array[i][0] = (
      valid_coord_array[i][0]/jpg_file.size[0]
  )
  valid_coord_array[i][1] = (
      valid_coord_array[i][1]/jpg_file.size[1]
  )
  valid_coord_array[i][2] = (
      valid_coord_array[i][2]/jpg_file.size[0]
  )
  valid_coord_array[i][3] = (
      valid_coord_array[i][3]/jpg_file.size[1]
  )

# изменение размера картинки и превращение её в массив numpy
  jpg_file_arr = img_to_array(jpg_file.copy().resize((256, 256)))

# заполнение массива для хранения валидационных изображений 256х256
  valid_img_arr[i] = xception.preprocess_input(jpg_file_arr)

In [ ]:
# загрузка и настройка модели Xception
# обучение и полносвязный слой для загруженной модели должны быть отключены
TL_model = Xception(
    weights = 'imagenet',
    input_shape = (256,256,3),
    include_top = False
)
TL_model.trainable = False

In [ ]:
# создание модели свёрточной нейронной сети
input_ = Input(shape=(256, 256, 3), name='images')
x = TL_model(input_, training = False)

x = Conv2D(kernel_size=3, strides=1, filters=128, padding='same',
                 activation='relu', name='layer_conv1')(x)

x = Flatten()(x)

# полносвязные слои для определения класса
x1 = Dense(128, activation = 'relu')(x)
x1 = Dropout(0.5)(x1)
x1 = Dense(64, activation = 'relu')(x1)

# полносвязные слои для определения координат
x2 = Dense(128, activation = 'relu')(x)
x2 = Dropout(0.5)(x2)
x2 = Dense(64, activation = 'relu')(x2)

# вывод класса
class_output = Dense(2, activation='softmax', name = 'class_output')(x1)

# вывод координат
bbox_output = Dense(4, activation='sigmoid', name = 'bbox_output')(x2)

model = tf.keras.models.Model(input_, [class_output, bbox_output])
model.summary()

In [ ]:
# метрика IoU
def iou_metric(y_true, y_pred):
    
    # площадь groundtruth box'а
    gt_area = K.abs(K.transpose(y_true)[2] - K.transpose(y_true)[0]
                ) * K.abs(K.transpose(y_true)[3] - K.transpose(y_true)[1])
    
    # площадь предсказанного box'а
    pred_area = K.abs(K.transpose(y_pred)[2] - K.transpose(y_pred)[0]
                ) * K.abs(K.transpose(y_pred)[3] - K.transpose(y_pred)[1])

    # координаты области пересечения
    xmin = K.maximum(K.transpose(y_true)[0], K.transpose(y_pred)[0])
    ymin = K.maximum(K.transpose(y_true)[1], K.transpose(y_pred)[1])
    xmax = K.maximum(K.transpose(y_true)[2], K.transpose(y_pred)[2])
    ymax = K.maximum(K.transpose(y_true)[3], K.transpose(y_pred)[3])

    # площадь пересечения
    intersection = (xmax - xmin) * (ymax - ymin)

    # площадь объединения
    union = gt_area + pred_area - intersection
    
    # вычисление IoU
    iou = intersection / union

    # привязать значения IoU к диапазону [0,1]
    iou = K.clip(iou, 0.0 + K.epsilon(), 1.0 - K.epsilon())

    return iou

In [ ]:
model.compile(
    loss={
        'class_output': 'categorical_crossentropy',
        'bbox_output': 'mse'
    },
    optimizer=tf.keras.optimizers.RMSprop(
        ),
    metrics={
        'class_output': 'accuracy',
        'bbox_output': [iou_metric]
    }
)

In [ ]:
model.fit(
    x = train_img_arr,
    y = {
        'class_output': train_class_array, 
        'bbox_output': train_coord_array
        },
    epochs=20,
    verbose = 1,
    validation_data = ({
        'images': valid_img_arr
    },
    {
        'class_output': valid_class_array,
        'bbox_output': valid_coord_array
    }),
)

In [ ]:
# вывод истинного и предсказанного bounding box'а на изображение
def plot_bounding_box(image, gt_coords, pred_coords = []):
  draw = ImageDraw.Draw(image)

# вывод истинного bounding box'а на изображение
  if len(gt_coords) == 4:
    xmin, ymin, xmax, ymax = gt_coords
    draw.rectangle((xmin, ymin, xmax, ymax), outline='green', width=3)

# вывод предсказанного bounding box'а на изображение
  if len(pred_coords) == 4:
    xmin, ymin, xmax, ymax = pred_coords
    draw.rectangle((xmin, ymin, xmax, ymax), outline='red', width=3)
  return image

In [ ]:
# выбрать номер картинки из массива
img_num = 0

# извлечь координаты предсказанного bounding box'а
pred_bbox = model.predict(valid_img_arr[img_num:img_num+1])[1][0]*256

# извлечь координаты истинного bounding box'а
gt_bbox = valid_coord_array[img_num]*256

# записать выбранную картинку в виде массива numpy
bb_img = array_to_img(valid_img_arr[img_num])

# нанести на картинку истинный и предсказанный bounding box
bb_img = plot_bounding_box(bb_img, gt_bbox, pred_bbox)

# вывести картинку с bounding box'ами на экран
plt.imshow(bb_img)

In [ ]:
import time

In [ ]:
# оценка inference time'а одной картинки
start = time.time()
model.predict(valid_img_arr[0:1])
end = time.time()
print(end - start)

In [ ]:
# оценка inference time'а десяти картинок и одной из них отдельно
# часть времени при предсказании затрачивается на сопутствующие вычисления
start = time.time()
model.predict(valid_img_arr[0:10])
end = time.time()
print(end - start)
print((end - start)/10)

In [ ]:
# 796s 8s/step 
#    - loss: 0.0034 - class_output_loss: 3.3889e-05 -     bbox_output_loss: 0.0034 -     class_output_accuracy: 1.0000 -     bbox_output_iou_metric: 0.9373 
#- val_loss: 0.0395 - val_class_output_loss: 0.0325 - val_bbox_output_loss: 0.0070 - val_class_output_accuracy: 0.9950 - val_bbox_output_iou_metric: 0.9217

In [ ]:
# 20 эпох, связки по 32 элемента
# точность классификации: 99.5%
# IoU: 92.17%
# inference time для 1 картинки: 0.34979748725891113, ~350 мс
# inference time для 10 картинок: 2.3396732807159424, ~2.34 с
# inference time для 1 картинки из 10: 0.23396732807159423, ~234 мс
# количество элементов в тренировочной выборке: 2985
# количество элементов в валидационной выборке: 400